In [2]:
#### Neuralopt experiments with softpoisson 
basestr = ["--experiment_type", "DEFAULT", "--size_train", "1", "--size_test", "1", "--epochs", "20000",
             "--val_interval", "50", "--align_2D", "--xp_type", "uv", "--gpu_strategy", "ddp",
              "--n_gpu", "1", "--no_poisson", "--noencoder",
              "--lossdistortion", "arap", "--projectname", "sp_neuralopt", "--lr", "1e-5",
              "--targets_per_batch", "16", "--workers", "0", "--valrenderratio", "1", 
              "--stitchingloss", "edgecutloss", "--accumulate_grad_batches", "1", "--outputdir", "./outputs/neuralopt/softpoisson",
              "--identity", "--facedim", "3", "--seamlessedgecut", "--seamlessdelta", "0.05", "--edgecut_weight", "1",
              "--spweight", "cosine", "--initweightinput", "--initjinput", 
              ]
basestr = "python ./training_scripts/train.py " + " ".join(basestr)

baseoutput = "./outputs/neuralopt/softpoisson"
datadirs = ['cylinder_nocut', 'cone', 'floatie', 'halfbunny', 'tetrahedron_sub1', 'tetrahedron_sub2']
softpoissons = ['edges']
inits = ['isometric']
stitchweights = ["softweightdetach", "stitchloss"]
sweight_schedule = ['linear', 'cosine']
spweights = ['cosine']
sparsepoisson = [True, False]

f = open("./slurm/softpoisson_nopt", 'w')
for data in datadirs: 
    expstr = basestr + f" --init isometric --ninit 1 --root_dir_train ./data/{data} --root_dir_test ./data/{data} --data_file {data}.json"
    expstr += f" --softpoisson edges"
    
    expname = f"neuralopt_{data}_initinputs"
    expstr += f" --expname {expname}"
    f.write(expstr + "\n")
f.close() 

In [2]:
#### Learning experiments with softpoisson 
basestr = ["--experiment_type", "DEFAULT", "--size_train", "1", "--size_test", "1", "--epochs", "10000",
             "--val_interval", "50", "--align_2D", "--xp_type", "uv", "--gpu_strategy", "ddp",
              "--n_gpu", "1", "--no_poisson", "--noencoder",
              "--lossdistortion", "arap", "--projectname", "softpoisson", "--lr", "1e-5",
              "--targets_per_batch", "16", "--workers", "0", "--facedim", "3", "--valrenderratio", "1", 
              "--stitchingloss", "edgecutloss", "--accumulate_grad_batches", "1", "--outputdir", "./outputs/learning/softpoisson",
              "--initweightinput", "--initjinput", 
              "--identity", "--seamlessedgecut", "--seamlessdelta", "0.05", "--edgecut_weight", "1",
              ]
basestr = "python ./training_scripts/train.py " + " ".join(basestr)

baseoutput = "./outputs/learning/softpoisson"
datadirs = ['cone', 'tetrahedron_sub2']
softpoissons = ['edges']
inits = ['tutte', 'slim', 'isometric']

f = open("./slurm/softpoisson_simple", 'w')
for data in datadirs: 
    for poisson in softpoissons:
        for init in inits: 
            expstr = basestr + f" --init {init} --ninit -1 --root_dir_train ./data/{data} --root_dir_test ./data/{data} --data_file {data}.json"
            expstr += f" --softpoisson {poisson} --simplecut"
            
            if data == "floatie":
                expstr += f" --min_cuts 2 --max_cuts 20"
            elif data == "cone":
                expstr += f" --min_cuts 0 --max_cuts 12"
            elif data == "tetrahedron_sub2":
                expstr += f" --min_cuts 0 --max_cuts 2"
            else:
                expstr += f" --min_cuts 0 --max_cuts 20"
            
            if init == "slim":
                expstr += f" --slimiters 2"
            elif init == "isometric":
                expstr += f" --basistype rot"
            
            expname = f"softpoiss{poisson}_weightinput_{data}_init{init}"
            expstr += f" --expname {expname}"
            f.write(expstr + "\n")
f.close() 

In [1]:
import os 

### Soft poisson direct opt 
basestr = "python opt_softpoisson.py "
datadirs = ['./data/tetrahedron4.obj', "./data/tetrahedron_sub1.obj", "./data/tetrahedron_sub2.obj"]
stitchweight = ['softweightdetach']
distortionlosses = ['symmetricdirichlet', 'arap']
edgecuts = ['seamless', 'vertexsep']
edgeweights = ['seploss', 'softweightdetach']

f = open("slurm/softpoisson_opt", 'w')
for data in datadirs: 
    for distortionloss in distortionlosses:
        for edgecut in edgecuts:
            for edgeweight in edgeweights:
                objname = os.path.basename(data).split(".")[0]
                savename = f"{objname}_ecut{edgecut}_eweight{edgeweight}_dist{distortionloss}"
                expstr = basestr + f" --objdir {data} --niters 300000 --init isometric --savename {savename} --lr 1e-4"
                expstr += f" --distortionloss {distortionloss} --edgeweight {edgeweight} --edgecutloss {edgecut}"
                expstr += f" --seplossdelta 0.01 --stitchweight softweightdetach " 
                expstr += f" --vertexseptype l2 --vertexsepsqrt"
            f.write(expstr + "\n")
f.close()

In [1]:
import os 

### Soft poisson direct opt 
basestr = "python opt_softpoisson.py "
datadirs = ['./data/tetrahedron4.obj', './data/tetrahedron5.obj', "./data/tetrahedron_sub1.obj", "./data/tetrahedron_sub2.obj"]
stitchweight = ['softweight', 'softweightdetach', 'seploss']
seamlesses = [True, False]
vseptype = ['l2sqrt']

f = open("slurm/softpoisson_opt", 'w')
for data in datadirs: 
    for seamless in seamlesses:
        for sweight in stitchweight:
            objname = os.path.basename(data).split(".")[0]
            savename = f"{objname}_edgecut_seamless{seamless}_sweight{sweight}"
            expstr = basestr + f" --objdir {data} --niters 300000 --init isometric --vertexseploss --symmetricdirichletloss --savename {savename} --lr 1e-4"
            expstr += f" --seplossdelta 0.01" 
            expstr += f" --vertexseptype l2 --vertexsepsqrt --edgecutloss seamless --stitchweight {sweight}"
            
            if seamless:
                expstr += f" --seamless"
            
            if sweight == "seploss": 
                expstr += f" --niters 1000000 --lr 1e-3"
                
            f.write(expstr + "\n")
f.close()

In [5]:
import os 

### Soft poisson direct opt 
basestr = "python opt_softpoisson.py "
datadirs = ['./data/tetrahedron.obj', './data/tetrahedron4.obj', "./data/tetrahedron_sub1.obj", "./data/tetrahedron_sub2.obj"]
weightlosses = [True, False]
seplossdeltas = [0.01, 0.001, 0.0001]
vseptype = ['l2sqrt', 'l2']

f = open("slurm/softpoisson_opt", 'w')
for data in datadirs: 
    for vsep in vseptype: 
        for delta in seplossdeltas:
            objname = os.path.basename(data).split(".")[0]
            savename = f"{objname}_septype{vsep}_sepdelta{delta}"
            expstr = basestr + f" --objdir {data} --niters 300000 --init isometric --vertexseploss --symmetricdirichletloss --savename {savename}"
            if vsep == 'l2sqrt':
                expstr += f" --lr 1e-4 --vertexseptype l2 --seamless --seplossdelta {delta} --vertexsepsqrt"
            else:
                expstr += f" --lr 1e-4 --vertexseptype {vsep} --seamless --seplossdelta {delta}"
            f.write(expstr + "\n")
f.close()

In [2]:
import os 

### Soft poisson direct opt 
basestr = "python opt_softpoisson.py "
datadirs = ['./data/floatie/floatie.obj', './data/halfbunny/halfbunny.obj', "./data/coarsecylinder_nocut/coarsecylinder_nocut.obj"]
weightlosses = [True, False]
seplossdeltas = [0.01, 0.001]

f = open("slurm/softpoisson_opt", 'w')
for data in datadirs: 
    for wloss in weightlosses:
        for seplossdelta in seplossdeltas:
            objname = os.path.basename(data).split(".")[0]
            savename = f"{objname}_wloss{wloss}_sepdelta{seplossdelta}"
            expstr = basestr + f" --objdir {data} --niters 300000 --init isometric --vertexseploss --symmetricdirichletloss --seplossdelta {seplossdelta} --savename {savename}"
            if wloss:
                expstr += f" --weightloss"
            f.write(expstr + "\n")

# Also try init tutte for coarsecylinder 
data = "./data/coarsecylinder_nocut/coarsecylinder_nocut.obj"
for wloss in weightlosses:
    for seplossdelta in seplossdeltas:
        objname = os.path.basename(data).split(".")[0]
        savename = f"{objname}_wloss{wloss}_sepdelta{seplossdelta}_initslim"
        expstr = basestr + f" --objdir {data} --niters 300000 --init slim --vertexseploss --symmetricdirichletloss --seplossdelta {seplossdelta} --savename {savename}"
        if wloss:
            expstr += f" --weightloss"
        f.write(expstr + "\n")
f.close()

In [13]:
#### Neural optimization: try cut cylinder with translation optimization (no random translation of isometric)
basestr = ["--experiment_type", "DEFAULT", "--size_train", "1", "--size_test", "1", "--epochs", "20000",
             "--val_interval", "20", "--align_2D", "--xp_type", "uv", "--gpu_strategy", "ddp",
              "--n_gpu", "1", "--no_poisson", "--identity", "--lr", "1e-5", "--ninit", "1", "--initjinput",
              "--targets_per_batch", "16", "--workers", "8", "--outputdir", "./outputs/neuralopt",
              "--accumulate_grad_batches", "1", "--projectname", "neuralopt"]
basestr = "python ./training_scripts/train.py " + ' '.join(basestr)
datadirs = ['cylinder']
l0relaxes = [True, False]
losses = ['cosine', 'l2', 'l1']

f = open("slurm/cylinder_trans", 'w')
for data in datadirs: 
    for l0 in l0relaxes:
        expname = f"trans_{data}_l0{l0}"
        expstr = basestr + f" --init isometric --lossedgeseparation --root_dir_train ./data/{data} --root_dir_test ./data/{data} --expname {expname}"
        expstr += f" --lossdistortion dirichlet --data_file {data}.json"
        if l0: 
            expstr += f" --seploss_schedule --stitchrelax"
        f.write(expstr + "\n")
f.close() 

In [14]:
#### Neural optimization: try different shapes
basestr = ["--experiment_type", "DEFAULT", "--size_train", "1", "--size_test", "1", "--epochs", "20000",
             "--val_interval", "20", "--align_2D", "--xp_type", "uv", "--gpu_strategy", "ddp",
              "--n_gpu", "1", "--no_poisson", "--identity", "--lr", "1e-5", "--ninit", "1", "--initjinput",
              "--targets_per_batch", "16", "--workers", "8", "--outputdir", "./outputs/neuralopt",
              "--accumulate_grad_batches", "1", "--opttrans", "--projectname", "neuralopt"]
basestr = "python ./training_scripts/train.py " + ' '.join(basestr)
inits = ['isometric', 'tutte']
datadirs = ['cylinder_nocut', "halfbunny", "cone", "floatie", "headphoneholder", "squirrel"]
distortions = ['dirichlet']
l0relaxes = [True, False]
losses = ['cosine', 'l2', 'l1']

f = open("slurm/neuralopt_exps", 'w')
for data in datadirs: 
    for l0 in l0relaxes:
        for loss in losses:
            expname = f"{data}_l0{l0}_loss{loss}"
            expstr = basestr + f" --init isometric --lossgradientstitching {loss} --root_dir_train ./data/{data} --root_dir_test ./data/{data} --expname {expname}"
            expstr += f" --lossdistortion dirichlet --data_file {data}.json"
            if l0: 
                expstr += f" --seploss_schedule --stitchrelax"
            f.write(expstr + "\n")
f.close() 

In [2]:
#### Coordinate MLP (GT loss)
basestr = ["--experiment_type", "DEFAULT", "--size_train", "1", "--size_test", "1", "--epochs", "10000",
             "--val_interval", "20", "--data_file", "cylinder.json", "--align_2D", "--xp_type", "uv", "--gpu_strategy", "ddp",
              "--n_gpu", "1", "--no_poisson", "--identity", "--initjinput", "--noencoder",
              "--projectname", "gtcyl", "--lr", "1e-4", "--lossgt",
              "--targets_per_batch", "16", "--workers", "1", "--outputdir", "./outputs/learning/gtcylinder",
              "--accumulate_grad_batches", "1", "--min_cuts", "2", "--max_cuts", "8",
              "--top_k_eig", "30"]

basestr = "python ./training_scripts/train.py " + ' '.join(basestr)
inits = ['tutte', 'isometric', 'slim']
isometric_bases = ['basis', 'rot']
slim_iters = [50, 100]
usedense = [True, False]
datadirs = ['coarsecylinder_nocut']

f = open("./slurm/gt_cylinder", 'w')
for data in datadirs: 
    for init in inits:
        expname = f"gtcyl_{data}_init{init}"
        if init == "isometric":
            for basis in isometric_bases:
                expname += f"_basis{basis}"
                expstr = basestr + f" --init {init} --ninit -1 --root_dir_train ./data/{data} --root_dir_test ./data/{data} --expname {expname} --basistype {basis}"
                f.write(expstr + "\n")
        elif init == "slim":
            for iter in slim_iters:
                expname += f"_iter{iter}"
                expstr = basestr + f" --init {init} --ninit -1 --root_dir_train ./data/{data} --root_dir_test ./data/{data} --expname {expname} --slimiters {iter}"
                f.write(expstr + "\n")
        else:
            expstr = basestr + f" --init {init} --ninit -1 --root_dir_train ./data/{data} --root_dir_test ./data/{data} --expname {expname}"
            f.write(expstr + "\n")
f.close() 

In [1]:
#### Coordinate MLP (SLIM cutting on bunny)
basestr = ["--experiment_type", "DEFAULT", "--size_train", "1", "--size_test", "1", "--epochs", "10000",
             "--val_interval", "20", "--align_2D", "--xp_type", "uv", "--gpu_strategy", "ddp",
              "--n_gpu", "1", "--no_poisson", "--identity", "--initjinput", "--noencoder",
              "--lossdistortion", "dirichlet", "--projectname", "cutlearning", "--lr", "1e-5",
              "--targets_per_batch", "16", "--workers", "1", "--outputdir", "./outputs/learning/halfbunny",
              "--accumulate_grad_batches", "1", "--opttrans", "--min_cuts", "10", "--max_cuts", "20",
              "--lossgradientstitching", "cosine"]

# TODO: opttrans only for gradient stitching -- debug the translation component for euclidean loss 
basestr = "python ./training_scripts/train.py " + ' '.join(basestr)
inits = ['tutte', 'slim']
slimiters = ["50", "100", "500"]
relaxstitches = [True, False]
losstype = ['gradient']
datadirs = ['halfbunny']
lrs = ['1e-5']

f = open("./slurm/bunny_mlp", 'w')
for data in datadirs: 
    for stitch in relaxstitches:
        for init in inits:
            expstr = basestr + f" --init {init} --ninit -1 --root_dir_train ./data/{data} --root_dir_test ./data/{data} --data_file {data}.json"
            if stitch: 
                expstr += " --stitchrelax"
                
            if init == "tutte":
                expname = f"coordmlp_{data}_init{init}_stitch{stitch}"
                expstr += f" --expname {expname}"
                
                f.write(expstr + "\n")
                
            elif init == "slim":
                for iter in slimiters:
                    expname = f"coordmlp_{data}_init{init}_stitch{stitch}_slim{iter}"
                    expstr += f" --slimiters {iter} --expname {expname}"
                    
                    f.write(expstr + "\n")
f.close() 

In [1]:
#### Dense MLP (Tutte cutting)
basestr = ["--experiment_type", "DEFAULT", "--size_train", "1", "--size_test", "1", "--epochs", "10000",
             "--val_interval", "20", "--data_file", "cylinder.json", "--align_2D", "--xp_type", "uv", "--gpu_strategy", "ddp",
              "--n_gpu", "1", "--no_poisson", "--identity", "--initjinput", "--noencoder",
              "--lossdistortion", "dirichlet", "--projectname", "densetutte", "--lr", "1e-5",
              "--targets_per_batch", "16", "--workers", "1", "--outputdir", "./outputs/learning/dense_tutte",
              "--accumulate_grad_batches", "1", "--opttrans", "--min_cuts", "1", "--max_cuts", "6",
              "--lossgradientstitching", "cosine", "--dense", "input",
              "--top_k_eig", "40"]

basestr = "python ./training_scripts/train.py " + ' '.join(basestr)
inits = ['tutte']
datadirs = ['coarsecylinder', 'coarsecylinder_nocut']
lrs = ['1e-5']

f = open("./slurm/cylinder_dense_tutte", 'w')
for data in datadirs: 
    for init in inits:
        expname = f"dense_{data}_init{init}"
        expstr = basestr + f" --init {init} --ninit -1 --root_dir_train ./data/{data} --root_dir_test ./data/{data} --expname {expname}"
        f.write(expstr + "\n")
f.close() 

In [2]:
#### Coordinate MLP (Tutte cutting)
basestr = ["--experiment_type", "DEFAULT", "--size_train", "1", "--size_test", "1", "--epochs", "10000",
             "--val_interval", "20", "--data_file", "cylinder.json", "--align_2D", "--xp_type", "uv", "--gpu_strategy", "ddp",
              "--n_gpu", "1", "--no_poisson", "--identity", "--initjinput", "--noencoder",
              "--lossdistortion", "dirichlet", "--projectname", "coarsetutte", "--lr", "1e-5",
              "--targets_per_batch", "16", "--workers", "1", "--outputdir", "./outputs/learning/coarse_tutte",
              "--accumulate_grad_batches", "1", "--opttrans", "--min_cuts", "1", "--max_cuts", "6",
              "--lossgradientstitching", "cosine",
              "--top_k_eig", "40"]

basestr = "python ./training_scripts/train.py " + ' '.join(basestr)
inits = ['tutte']
datadirs = ['coarsecylinder', 'coarsecylinder_nocut']
lrs = ['1e-5']

f = open("./slurm/cylinder_coarse_tutte", 'w')
for data in datadirs: 
    for init in inits:
        expname = f"coordmlp_{data}_init{init}"
        expstr = basestr + f" --init {init} --ninit -1 --root_dir_train ./data/{data} --root_dir_test ./data/{data} --expname {expname}"
        f.write(expstr + "\n")
f.close() 

In [1]:
#### Dense MLP
## data type 
## stitching type 
## global rotation sampling/random/basis sampling 
basestr = ["--experiment_type", "DEFAULT", "--size_train", "1", "--size_test", "1", "--epochs", "10000",
             "--val_interval", "20", "--data_file", "cylinder.json", "--align_2D", "--xp_type", "uv", "--gpu_strategy", "ddp",
              "--n_gpu", "1", "--no_poisson", "--identity", "--initjinput", "--noencoder",
              "--lossdistortion", "dirichlet", "--projectname", "dense", "--lr", "1e-5",
              "--targets_per_batch", "16", "--workers", "1", "--outputdir", "./outputs/dense_v2",
              "--accumulate_grad_batches", "1", "--opttrans",
              "--dense", "input", "--lossgradientstitching", "cosine",
              "--top_k_eig", "40"]

basestr = "python ./training_scripts/train.py " + ' '.join(basestr)
inits = ['isometric']
basistypes = ['rot', 'basis', 'global']
gradclips = [True, False]
datadirs = ['coarsecylinder', 'coarsecylinder_nocut']
lrs = ['1e-5']

f = open("./slurm/cylinder_dense", 'w')
for data in datadirs: 
    for init in inits:
        for basis in basistypes: 
            expname = f"{data}_init{init}_basis{basis}"
            expstr = basestr + f" --init {init} --ninit -1 --root_dir_train ./data/{data} --root_dir_test ./data/{data} --expname {expname}"
            expstr += f" --basistype {basis}"
            f.write(expstr + "\n")
f.close() 

In [2]:
#### Coordinate MLP
## data type 
## stitching type 
## global rotation sampling/random/basis sampling 
basestr = ["--experiment_type", "DEFAULT", "--size_train", "1", "--size_test", "1", "--epochs", "10000",
             "--val_interval", "20", "--data_file", "cylinder.json", "--align_2D", "--xp_type", "uv", "--gpu_strategy", "ddp",
              "--n_gpu", "1", "--no_poisson", "--identity", "--initjinput", "--noencoder",
              "--lossdistortion", "dirichlet", "--projectname", "dense", "--lr", "1e-5",
              "--targets_per_batch", "16", "--workers", "1", "--outputdir", "./outputs/coordmlp",
              "--accumulate_grad_batches", "1", "--opttrans",
              "--lossgradientstitching", "cosine",
              "--top_k_eig", "40"]

basestr = "python ./training_scripts/train.py " + ' '.join(basestr)
inits = ['isometric']
basistypes = ['rot', 'basis', 'global']
gradclips = [True, False]
datadirs = ['coarsecylinder', 'coarsecylinder_nocut']
lrs = ['1e-5']

f = open("./slurm/cylinder_coarse", 'w')
for data in datadirs: 
    for init in inits:
        for basis in basistypes: 
            expname = f"coordmlp_{data}_init{init}_basis{basis}"
            expstr = basestr + f" --init {init} --ninit -1 --root_dir_train ./data/{data} --root_dir_test ./data/{data} --expname {expname}"
            expstr += f" --basistype {basis}"
            f.write(expstr + "\n")
f.close() 

In [7]:
#### Hyperparam search: neural optimization w/ initj input
## - distortion loss: arap/dirichlet 
## - initialization: tutte, isometric 
## - stitchloss schedule: T/F (tutte: 0->1, isometric: 1->2)
## - l0relaxation + schedule: T/F

basestr = ["--experiment_type", "DEFAULT", "--size_train", "1", "--size_test", "1", "--epochs", "20000",
             "--val_interval", "20", "--data_file", "cylinder.json", "--align_2D", "--xp_type", "uv", "--gpu_strategy", "ddp",
              "--n_gpu", "1", "--no_poisson", "--identity", "--lr", "1e-5", "--ninit", "1", "--initjinput",
              "--targets_per_batch", "16", "--workers", "8", "--outputdir", "./outputs/neuralopt",
              "--accumulate_grad_batches", "1", "--opttrans"]
basestr = "python ./training_scripts/train.py " + ' '.join(basestr)
inits = ['isometric', 'tutte']
datadirs = ['cylinder', 'cylinder_nocut']
distortions = ['dirichlet', 'arap']
stitchschedules = [True, False]
l0relaxes = [True, False]

f = open("slurm/cylinder_neuralopt", 'w')
for data in datadirs: 
    for distortion in distortions: 
        for init in inits:
            for stitchschedule in stitchschedules:
                for l0 in l0relaxes:
                    expname = f"{data}_dist{distortion}_init{init}_stitchsched{stitchschedule}_l0{l0}"
                    expstr = basestr + f" --init {init} --lossgradientstitching l2 --root_dir_train ./data/{data} --root_dir_test ./data/{data} --expname {expname}"
                    expstr += f" --lossdistortion {distortion}"
                    if stitchschedule: 
                        if init == "isometric":
                            expstr += f" --stitchloss_schedule linear --stitchlossweight_min 1 --stitchlossweight_max 2"
                        if init == "tutte":
                            expstr += f" --stitchloss_schedule linear --stitchlossweight_min 0 --stitchlossweight_max 1"
                    if l0: 
                        expstr += f" --seploss_schedule --stitchrelax"
                    f.write(expstr + "\n")
f.close() 

In [3]:
# Multiple init learning experiment  
basestr = ["--experiment_type", "DEFAULT", "--size_train", "1", "--size_test", "1", "--epochs", "10000",
             "--val_interval", "50", "--data_file", "cylinder.json", "--align_2D", "--xp_type", "uv", "--gpu_strategy", "ddp",
              "--n_gpu", "1", "--no_poisson", "--identity", "--lr", "1e-5", "--workers", "8", "--opttrans",
              "--lossdistortion", "dirichlet", "--targets_per_batch", "16", "--cuteps", "0.5",  
              "--accumulate_grad_batches", "1", "--initjinput", "--outputdir", "outputs/learning"]

basestr = "python ./training_scripts/train.py " + ' '.join(basestr)
inits = ['isometric']
losses = [None, 'l2']
basistypes = ['basis', 'rot']
# ninits = [1,2,5,10,20,100]
ninit_to_ratio = {1:1, 2:1, 5:1, 10: 0.2, 20: 0.1, 100: 0.1}
datadirs = ['cylinder', 'cylinder_nocut']

f = open("slurm/cylinder_n", 'w')
for data in datadirs: 
    for init in inits:
        for loss in losses:
            for basis in basistypes: 
                expname = f"{data}_init{init}_loss{loss}_basis{basis}"
                expstr = basestr + f" --init {init} --ninit -1 --root_dir_train ./data/{data} --root_dir_test ./data/{data} --expname {expname}"
                expstr += f" --basistype {basis}"
                # expstr += f" --valrenderratio {ninit_to_ratio[ninit]}"
                if loss:
                    expstr += f" --lossgradientstitching {loss}"
                else:
                    expstr += f" --lossedgeseparation"
                f.write(expstr + "\n")
f.close() 

In [2]:
# Gradient stitching experiments 
basestr = ["--experiment_type", "DEFAULT", "--size_train", "1", "--size_test", "1", "--epochs", "3000",
             "--val_interval", "20", "--data_file", "cylinder.json", "--align_2D", "--xp_type", "uv", "--gpu_strategy", "ddp",
              "--n_gpu", "1", "--no_poisson", "--identity", "--lr", "0.0001",
              "--targets_per_batch", "16",
              "--accumulate_grad_batches", "1"]
basestr = "python ./training_scripts/train.py " + ' '.join(basestr)
# ffts = [True, False]
inits = ['isometric', 'tutte']
losses = ['l2']
datadirs = ['cylinder', 'cylinder_nocut']
distortions = ['dirichlet', 'edge']

f = open("slurm/cylinder_grad_exps", 'w')
for data in datadirs: 
    for distortion in distortions: 
        for init in inits:
            for loss in losses:
                expname = f"{data}_dist{distortion}_init{init}_loss{loss}"
                expstr = basestr + f" --init {init} --lossgradientstitching {loss} --root_dir_train ./data/{data} --root_dir_test ./data/{data} --expname {expname}"
                # if fft:
                #     expstr += " --fft --fft_dim 256"
                expstr += f" --lossdistortion {distortion}"
                f.write(expstr + "\n")
f.close() 

In [5]:
# Flat direct optimization experiments
basestr = "python directopt.py"

datadirs = ['cyl', 'cylnocut']
inits = ['isom']
# anneals = [True, False]
grads = [None, 'l2', 'split']

f = open("slurm/directopt_tests_v2", 'w')
for data in datadirs:
    for init in inits:
        for grad in grads:
            # Don't anneal if grad
            if anneal and grad is not None: 
                continue 
            expname = f"dopt_v2_data{data}_init{init}_grad{grad}"
            expstr = basestr + f" --vs ./scratch/data/{data}_vs.pt --fs ./scratch/data/{data}_fs.pt --init ./scratch/data/{data}_{init}.pt --savedir ./scratch/{expname}"
            if grad:
                expstr += f" --grad {grad}"
            f.write(expstr + "\n")
f.close() 


In [3]:
# Flat sanity check experiments
basestr = ["--experiment_type", "DEFAULT", "--size_train", "1", "--size_test", "1", "--epochs", "2000",
             "--val_interval", "20", "--data_file", "cylinder.json", "--align_2D", "--xp_type", "uv", "--gpu_strategy", "ddp",
              "--n_gpu", "1", "--no_poisson", "--identity", "--lr", "1e-5",
              "--layer_normalization", "FLATTEN",
              "--lossdistortion", "dirichlet", "--targets_per_batch", "16",
              "--accumulate_grad_batches", "1"]
basestr = "python ./training_scripts/train.py " + ' '.join(basestr)

datadirs = ['cylinder', 'cylinder_nocut']
inits = ['isometric', 'tutte']
sepdeltas = [0.1]
# schedules = [True, False]
grads = [None, 'l2', 'split']

f = open("slurm/cylinder_flat", 'w')
for data in datadirs:
    for delta in sepdeltas:
        for init in inits:
            for grad in grads:
                expname = f"flatcyl_sepd{delta}_data{data}_init{init}_grad{grad}"
                expstr = basestr + f" --root_dir_train ./data/{data} --root_dir_test ./data/{data} --init {init} --seplossdelta {delta} --expname {expname}"
                if grad:
                    expstr += f" --lossgradientstitching {grad}"
                else: 
                    expstr += f" --lossedgeseparation --seplossweight 1"
                f.write(expstr + "\n")
f.close() 


In [3]:
basestr = ["--root_dir_train", "./data/cylinder", "--root_dir_test", "./data/cylinder",
             "--experiment_type", "DEFAULT", "--size_train", "1", "--size_test", "1", "--epochs", "8000",
             "--val_interval", "20", "--data_file", "cylinder.json", "--align_2D", "--xp_type", "uv", "--gpu_strategy", "ddp",
              "--n_gpu", "1", "--no_poisson", "--identity", "--lr", "0.005",
              "--lossdistortion", "dirichlet", "--lossedgeseparation", "--seplossweight", "1", "--targets_per_batch", "16",
              "--accumulate_grad_batches", "1", "--fft", "--fft_dim", "256"]
basestr = "python ./training_scripts/train.py " + ' '.join(basestr)
sepdeltas = [0.01, 0.1, 0.5]
seplosses = ['l1', 'mse']

f = open("slurm/cylinder_exps", 'w')
for delta in sepdeltas:
    for seploss in seplosses:
        expname = f"cylinder_sepd{delta}_sepl{seploss}"
        expstr = basestr + f" --init isometric --eseploss {seploss} --seplossdelta {delta} --expname {expname}"
        f.write(expstr + "\n")
f.close() 

inits = ['isometric', 'tutte']
f = open("slurm/cylinder_exps_v2", 'w')
for init in inits:
    for seploss in seplosses:
        expname = f"cylinder_init{init}_sepl{seploss}"
        expstr = basestr + f" --init {init} --eseploss {seploss} --expname {expname} --seploss_schedule --seplossdelta_min 0.001"
        f.write(expstr + "\n")
f.close() 

# Same for bunny
basestr = ["--root_dir_train", "./data/bunny", "--root_dir_test", "./data/bunny",
             "--experiment_type", "DEFAULT", "--size_train", "1", "--size_test", "1", "--epochs", "8000",
             "--val_interval", "20", "--data_file", "bunny.json", "--align_2D", "--xp_type", "uv", "--gpu_strategy", "ddp",
              "--n_gpu", "1", "--no_poisson", "--identity", "--init", "isometric", "--lr", "0.005",
              "--lossdistortion", "dirichlet", "--lossedgeseparation", "--seplossweight", "1", "--targets_per_batch", "16",
              "--accumulate_grad_batches", "1"]
basestr = "python ./training_scripts/train.py " + ' '.join(basestr)
sepdeltas = [0.01, 0.1, 0.5]
seplosses = ['l1', 'mse']

f = open("slurm/bunny_exps", 'w')
for delta in sepdeltas:
    for seploss in seplosses:
        expname = f"bunny_sepd{delta}_sepl{seploss}"
        expstr = basestr + f" --eseploss {seploss} --seplossdelta {delta} --expname {expname}"
        f.write(expstr + "\n")
f.close() 

In [1]:
argstr = """--root_dir_train ./data/cylinder --root_dir_test ./data/cylinder --expname Debug --experiment_type DEFAULT 
        --size_train 1 --size_test 1 --epochs 5000 --val_interval 20 --data_file cylinder.json --align_2D --xp_type uv 
        --gpu_strategy ddp --n_gpu 1 --no_poisson --lossdistortion dirichlet --identity --init isometric --lossedgeseparation 
        --seplossdelta 0.1 --seplossweight 1 --targets_per_batch 16 --accumulate_grad_batches 1"""
print(" ".join(argstr.split()))

--root_dir_train ./data/cylinder --root_dir_test ./data/cylinder --expname Debug --experiment_type DEFAULT --size_train 1 --size_test 1 --epochs 5000 --val_interval 20 --data_file cylinder.json --align_2D --xp_type uv --gpu_strategy ddp --n_gpu 1 --no_poisson --lossdistortion dirichlet --identity --init isometric --lossedgeseparation --seplossdelta 0.5 --seplossweight 1 --targets_per_batch 16 --accumulate_grad_batches 1
